In [1]:
import requests
import json, ndjson
from collections import Counter

with open("secret.csv", "r") as r:
    data = [line.replace("\n","").split(",") for line in r.readlines()]
    username = data[0][1]
    token = data[1][1]

In [6]:
url = "https://lichess.org"
print(f"Gathering data for {username}")
response = requests.get(
  f'https://lichess.org/api/games/user/{username}',
  params={
    "perfType" : "blitz",
    "rated" :  "true",
    "opening" : "true",
    "moves" : "true"
  },
  headers={
    'Authorization': f'Bearer {token}', # Need this or you will get a 401: Not Authorized response
    "Accept": "application/x-ndjson"
  }
)

Gathering data for sirsnakerb


In [7]:
# Parse application/x-ndjson into list of JSON objects
games = []
ndjson = response.content.decode().split('\n')

for json_obj in ndjson:
    if json_obj:
        games.append(json.loads(json_obj))

In [8]:
len(games)

3695

In [212]:
filtered_games = []
for game in games:
    if game.get("winner"):
        unw = game.get("players").get("white").get("user").get("name")
        unb = game.get("players").get("black").get("user").get("name")
        if (game.get("winner") == "white" and username == unw) or (game.get("winner") == "black" and username == unb):
            conclusion = "won"
        else:
            conclusion = "lost"
    else:
        conclusion = "tie"

    filtered_game = {
        "id" : game["id"],
        "moves" : game["moves"].split(" "),
        "outcome" : conclusion
    } 
    filtered_games.append(filtered_game) 

In [213]:
depth = 8
tree = {}

for i in range(1, depth):
    for game in filtered_games:
        moves = game.get("moves")[:i]
        if len(moves) == i: #else it will overwrite branches with short games
            set_by_path(tree, moves, {"score" : {
                    "won" : 0,
                    "lost" : 0,
                    "tie" : 0
            }})
            

for i in range(1, depth):
    for game in filtered_games:
        moves = game.get("moves")[:i]
        moves.append("score")
        moves.append(game["outcome"])
        set_by_path(tree, moves, get_by_path(tree, moves) + 1)

In [216]:
tree

{'e4': {'score': {'won': 1003, 'lost': 1150, 'tie': 120},
  'd5': {'score': {'won': 185, 'lost': 228, 'tie': 15},
   'exd5': {'score': {'won': 67, 'lost': 112, 'tie': 8},
    'Nf6': {'score': {'won': 1, 'lost': 5, 'tie': 1},
     'c4': {'score': {'won': 0, 'lost': 3, 'tie': 0},
      'c6': {'score': {'won': 0, 'lost': 1, 'tie': 0},
       'Nc3': {'score': {'won': 0, 'lost': 1, 'tie': 0}}},
      'e6': {'score': {'won': 0, 'lost': 1, 'tie': 0},
       'Nc3': {'score': {'won': 0, 'lost': 1, 'tie': 0}}},
      'Bg4': {'score': {'won': 0, 'lost': 1, 'tie': 0},
       'f3': {'score': {'won': 0, 'lost': 1, 'tie': 0}}}},
     'Nc3': {'score': {'won': 1, 'lost': 1, 'tie': 1},
      'Nxd5': {'score': {'won': 0, 'lost': 1, 'tie': 1},
       'Bc4': {'score': {'won': 0, 'lost': 1, 'tie': 1}}},
      'e6': {'score': {'won': 1, 'lost': 0, 'tie': 0},
       'dxe6': {'score': {'won': 1, 'lost': 0, 'tie': 0}}}},
     'Bc4': {'score': {'won': 0, 'lost': 1, 'tie': 0},
      'Bf5': {'score': {'won': 0, 'l

In [74]:
from functools import reduce  
import operator

def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    return reduce(operator.getitem, items, root)

def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value